# Google ML Sandbox Project

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.learn.python.learn import learn_io
import math
tf.logging.set_verbosity(tf.logging.ERROR)

import importlib.util
spec = importlib.util.spec_from_file_location("gml_util.name", "source/gml_util.py")
util = importlib.util.module_from_spec(spec)
spec.loader.exec_module(util)

%matplotlib inline

## 1. Import Data

In [18]:
raw_data = pd.read_csv(util.movie_data_path, sep=',', quotechar='"')
raw_data.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4993.000000,5028.000000,4939.000000,5020.000000,5036.000000,4.159000e+03,5.043000e+03,5043.000000,5030.000000,5022.000000,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
mean,140.194272,107.201074,686.509212,645.009761,6560.047061,4.846841e+07,8.366816e+04,9699.063851,1.371173,272.770808,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,121.601675,25.197441,2813.328607,1665.041728,15020.759120,6.845299e+07,1.384853e+05,18163.799124,2.013576,377.982886,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,50.000000,93.000000,7.000000,133.000000,614.000000,5.340988e+06,8.593500e+03,1411.000000,0.000000,65.000000,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,110.000000,103.000000,49.000000,371.500000,988.000000,2.551750e+07,3.435900e+04,3090.000000,1.000000,156.000000,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,195.000000,118.000000,194.500000,636.000000,11000.000000,6.230944e+07,9.630900e+04,13756.500000,2.000000,326.000000,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.500000,16.000000,349000.000000


## 2. Data Overview

In [32]:
util.print_data_shape(raw_data)

Data shape: (5043, 28)


In [28]:
for idx, feature_name in enumerate(raw_data.columns):
    print('{0:2}: {1}'.format(idx, feature_name))

 0: color
 1: director_name
 2: num_critic_for_reviews
 3: duration
 4: director_facebook_likes
 5: actor_3_facebook_likes
 6: actor_2_name
 7: actor_1_facebook_likes
 8: gross
 9: genres
10: actor_1_name
11: movie_title
12: num_voted_users
13: cast_total_facebook_likes
14: actor_3_name
15: facenumber_in_poster
16: plot_keywords
17: movie_imdb_link
18: num_user_for_reviews
19: language
20: country
21: content_rating
22: budget
23: title_year
24: actor_2_facebook_likes
25: imdb_score
26: aspect_ratio
27: movie_facebook_likes


## 3. Data Cleaning

Shuffle data:

In [49]:
data = raw_data.reindex(np.random.permutation(raw_data.index))

Clean inf and nan data points:

In [43]:
data = util.clean(data)
util.print_data_shape(data, 'Cleaned data')

Cleaned data shape: (4935, 28)


As mentioned in Important notes, point #6, some gross are denominated in other currencies to keep things simple, let's focus on American films => left with 3733 rows

In [45]:
data = util.get_us_only(data)
util.print_data_shape(data, 'USA only movie data')

USA only movie data shape: (3733, 28)


As mentioned in Important notes, point #3, there are around 800 "0"s in the "gross" attribute, remove those rows.

In [48]:
data = util.get_does_have_gross(data)
util.print_data_shape(data, 'Has gross record data')

Has gross record data shape: (3235, 28)


## 4. Cleaned Data Overview

In [50]:
data['title_year'].describe()

count    4935.000000
mean     2002.470517
std        12.474599
min      1916.000000
25%      1999.000000
50%      2005.000000
75%      2011.000000
max      2016.000000
Name: title_year, dtype: float64

In [13]:
clean_df['gross'].describe()

count    3.235000e+03
mean     5.521461e+07
std      7.173312e+07
min      7.030000e+02
25%      1.011027e+07
50%      3.217878e+07
75%      7.214700e+07
max      7.605058e+08
Name: gross, dtype: float64

In [14]:
# normalize gross, i.e. bring all gross revenue to 2016 assuming 4% annual inflation since 1920
# the reality is more complicated https://inflationdata.com/Inflation/Inflation_Rate/Long_Term_Inflation.asp
# but I have to start somewhere... i.e. 4%

clean_df['adjusted_profit'] = (clean_df['gross'] - clean_df['budget']) * 1.04**(2016 - clean_df['title_year'])
clean_df['adjusted_profit'].describe()

count    3.074000e+03
mean     4.048499e+07
std      1.645230e+08
min     -2.631049e+08
25%     -1.379435e+07
50%      5.558011e+06
75%      4.994008e+07
max      4.054227e+09
Name: adjusted_profit, dtype: float64

In [16]:
# print out the top 15 in terms of adjusted_profit for sanity checking
sorted = clean_df.sort_values(by='adjusted_profit', ascending=False, axis=0).loc[:,['gross', 'adjusted_profit',
       'movie_title', 'plot_keywords', 'language', 'country', 'budget', 'title_year', 'imdb_score']]
sorted.head(15)

,gross,adjusted_profit,movie_title,plot_keywords,language,country,budget,title_year,imdb_score
4449,184925485.0,4.054227e+09,Snow White and the Seven Dwarfs,dwarf|love|princess|queen|snow white,English,USA,2000000.0,1937.0,7.7
3970,198655278.0,3.989189e+09,Gone with the Wind,battle of the sexes|gallantry|honesty|indiffer...,English,USA,3977000.0,1939.0,8.2
3024,460935665.0,2.077068e+09,Star Wars: Episode IV - A New Hope,death star|empire|galactic war|princess|rebellion,English,USA,11000000.0,1977.0,8.7
3080,434949459.0,1.610496e+09,E.T. the Extra-Terrestrial,bicyclist|boy|doll|star wars reference|voice i...,English,USA,10500000.0,1982.0,7.9
1143,84300000.0,1.609740e+09,Pinocchio,boy|conscience|fairy|jiminy cricket|pinocchio,English,USA,2600000.0,1940.0,7.5
4225,76400000.0,1.460391e+09,Fantasia,apprentice|classical music|disney|mouse|orchestra,English,USA,2280000.0,1940.0,7.8
2917,260000000.0,1.258251e+09,Jaws,beach|blockbuster|fishing|great white shark|shark,English,USA,8000000.0,1975.0,8.0
3359,163214286.0,1.145703e+09,The Sound of Music,austria|children|governess|love|orchestral mus...,English,USA,8200000.0,1965.0,8.0
2051,290158751.0,1.116921e+09,Star Wars: Episode V - The Empire Strikes Back,duel|famous twist|rebel|rescue|snowy landscape,English,USA,18000000.0,1980.0,8.8
2916,204565000.0,1.061548e+09,The Exorcist,demonic possession|exorcism|exorcist|loss of i...,English,USA,8000000.0,1973.0,8.0


In [15]:
clean_df.shape

(3235, 29)

In [16]:
correlation_df = clean_df.copy()
correlation_df.corr()['adjusted_profit'].sort_values()

title_year                  -0.375180
aspect_ratio                -0.086631
facenumber_in_poster        -0.031801
budget                      -0.031506
actor_1_facebook_likes       0.006957
cast_total_facebook_likes    0.025583
actor_2_facebook_likes       0.033938
actor_3_facebook_likes       0.057017
director_facebook_likes      0.082494
movie_facebook_likes         0.086606
num_critic_for_reviews       0.092278
duration                     0.140801
num_user_for_reviews         0.234989
imdb_score                   0.259685
num_voted_users              0.335221
gross                        0.515573
adjusted_profit              1.000000
Name: adjusted_profit, dtype: float64

In [17]:
correlation_df.columns

Index([u'color', u'director_name', u'num_critic_for_reviews', u'duration',
       u'director_facebook_likes', u'actor_3_facebook_likes', u'actor_2_name',
       u'actor_1_facebook_likes', u'gross', u'genres', u'actor_1_name',
       u'movie_title', u'num_voted_users', u'cast_total_facebook_likes',
       u'actor_3_name', u'facenumber_in_poster', u'plot_keywords',
       u'movie_imdb_link', u'num_user_for_reviews', u'language', u'country',
       u'content_rating', u'budget', u'title_year', u'actor_2_facebook_likes',
       u'imdb_score', u'aspect_ratio', u'movie_facebook_likes',
       u'adjusted_profit'],
      dtype='object')

In [18]:
"""
these features are available before the movie comes out

title_year                  -0.375180
aspect_ratio                -0.086631
facenumber_in_poster        -0.031801
budget                      -0.031506
actor_1_facebook_likes       0.006957
cast_total_facebook_likes    0.025583
actor_2_facebook_likes       0.033938
actor_3_facebook_likes       0.057017
director_facebook_likes      0.082494
movie_facebook_likes         0.086606
duration                     0.140801
"""

'\nthese features are available before the movie comes out\n\ntitle_year                  -0.375180\naspect_ratio                -0.086631\nfacenumber_in_poster        -0.031801\nbudget                      -0.031506\nactor_1_facebook_likes       0.006957\ncast_total_facebook_likes    0.025583\nactor_2_facebook_likes       0.033938\nactor_3_facebook_likes       0.057017\ndirector_facebook_likes      0.082494\nmovie_facebook_likes         0.086606\nduration                     0.140801\n'

In [19]:
def preprocess_features(input_df):
    """Prepares input features from data set.
    Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
    """
    selected_features = input_df[['duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 
       'actor_1_facebook_likes', 
       'cast_total_facebook_likes',
       'facenumber_in_poster',
       'title_year', 'actor_2_facebook_likes']]
    processed_features = selected_features.copy()
    
    # Create a synthetic feature?
    
    return processed_features


def preprocess_targets(input_df):
    """Prepares target features (i.e., labels)
    Returns:
    A DataFrame that contains the target feature.
    """
    output_targets = pd.DataFrame()
    
    # Scale the target to be in units of millions of dollars.
    output_targets["adjusted_profit"] = (input_df["adjusted_profit"] / 1000000.0)
    return output_targets

In [24]:
clean_df = clean_df.reindex(np.random.permutation(clean_df.index))
filled_df = clean_df.fillna(0)
x = preprocess_features(filled_df.iloc[0:2000,:])
y = preprocess_targets(filled_df.iloc[0:2000,:])

In [22]:
def train_model(
    target,
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    """Trains a linear regression model.

    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.

    Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    ...

    Returns:
    A `LinearRegressor` object trained on the training data.
    """

    periods = 10
    steps_per_period = steps / periods

    # Create a linear regressor object.
    feature_columns = set([tf.contrib.layers.real_valued_column(my_feature) for my_feature in training_examples])
    linear_regressor = tf.contrib.learn.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate),
      gradient_clip_norm=5.0
    )

    # Create input functions
    training_input_fn = learn_io.pandas_input_fn(
      x=training_examples, y=training_targets[target],
      num_epochs=None, batch_size=batch_size)
    predict_training_input_fn = learn_io.pandas_input_fn(
      x=training_examples, y=training_targets[target],
      num_epochs=1, shuffle=False)
    predict_validation_input_fn = learn_io.pandas_input_fn(
      x=validation_examples, y=validation_targets[target],
      num_epochs=1, shuffle=False)

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("RMSE (on training data):")
    training_rmse = []
    validation_rmse = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        linear_regressor.fit(
            input_fn=training_input_fn,
            steps=steps_per_period,
        )
        # Take a break and compute predictions.
        training_predictions = list(linear_regressor.predict(input_fn=predict_training_input_fn))
        validation_predictions = list(linear_regressor.predict(input_fn=predict_validation_input_fn))
        # Compute training and validation loss.
        training_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(training_predictions, training_targets))
        validation_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(validation_predictions, validation_targets))
        # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
        # Add the loss metrics from this period to our list.
        training_rmse.append(training_root_mean_squared_error)
        validation_rmse.append(validation_root_mean_squared_error)
    print("Model training finished.")


    # Output a graph of loss metrics over periods.
    plt.ylabel("RMSE")
    plt.xlabel("Periods")
    plt.title("Root Mean Squared Error vs. Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label="training")
    plt.plot(validation_rmse, label="validation")
    plt.legend()

    return linear_regressor

In [23]:
train_model(
    target='adjusted_profit',
    learning_rate=0.00005,
    steps=1000,
    batch_size=20,
    training_examples=x,
    training_targets=y,
    validation_examples=preprocess_features(filled_df.iloc[2000:,:]),
    validation_targets=preprocess_targets(filled_df.iloc[2000:,:]))

Training model...
RMSE (on training data):
  period 00 : 181.30
  period 01 : 179.55
  period 02 : 179.07
  period 03 : 178.92
  period 04 : 178.54
  period 05 : 179.24
  period 06 : 181.62
  period 07 : 178.40
  period 08 : 178.33
  period 09 : 179.63
Model training finished.


LinearRegressor(params={'gradient_clip_norm': 5.0, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x11a6a0e10>, 'joint_weights': False, 'optimizer': <tensorflow.python.training.gradient_descent.GradientDescentOptimizer object at 0x11a6a0e50>, 'feature_columns': set([_RealValuedColumn(column_name='cast_total_facebook_likes', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='actor_2_facebook_likes', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='facenumber_in_poster', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='title_year', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='actor_1_facebook_likes', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='duration', dimension=1, default_value=None, dt